In [1]:
import numpy as np
import pandas as pd
import os
import sys
sys.path.append("../")
from utils.helper_function import *

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score

import optuna
import pickle
import datetime

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning, module="pandas")
warnings.filterwarnings("ignore", category=UserWarning)


/tmp/ipykernel_29151/240062032.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ModuleNotFoundError: No module named 'utils'

In [ ]:
xls = pd.ExcelFile('../data/data_sixpk.xlsx')
df = pd.read_excel(xls, 'SkinIrritation')
df.head()

,CASRN,SMILES,Activity
0,100-02-7,OC1=CC=C(C=C1)[N+]([O-])=O,1
1,100-18-5,CC(C)C1=CC=C(C=C1)C(C)C,1
2,100-20-9,ClC(=O)C1=CC=C(C=C1)C(Cl)=O,1
3,100-36-7,CCN(CC)CCN,1
4,100-37-8,CCN(CC)CCO,1


In [ ]:
X,y = df["SMILES"], df['Activity']

In [ ]:
idx = valid_indices(X)
X = X[idx]
y = y[idx]

In [ ]:
maccs = maccs_fp(X)


In [ ]:
maccs = np.array(maccs)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(maccs, y, test_size= 0.2, random_state= 92)

In [ ]:
def svm_modl(trial, xtrain, ytrain):
    n = trial.suggest_int('degree', 2, 5)
    model_svm = SVC(degree= n)

    scores = cross_validate(model_svm, xtrain, ytrain, cv=5, scoring='roc_auc')
    mean_roc = scores['test_score'].mean()

    return 1/(mean_roc + 1e-6)

In [ ]:
study2 = optuna.create_study(study_name='SVM_study_skin_irritation', storage= None)  # Create a new study.
study2.optimize(lambda trial: svm_modl(trial, X_train, y_train), n_trials=20)  # Invoke optimization of the objective function.

[I 2024-01-29 17:02:14,275] A new study created in memory with name: SVM_study_skin_irritation
[I 2024-01-29 17:02:16,280] Trial 0 finished with value: 1.2185334046006837 and parameters: {'degree': 4}. Best is trial 0 with value: 1.2185334046006837.
[I 2024-01-29 17:02:18,237] Trial 1 finished with value: 1.2185334046006837 and parameters: {'degree': 3}. Best is trial 0 with value: 1.2185334046006837.
[I 2024-01-29 17:02:20,156] Trial 2 finished with value: 1.2185334046006837 and parameters: {'degree': 4}. Best is trial 0 with value: 1.2185334046006837.
[I 2024-01-29 17:02:22,098] Trial 3 finished with value: 1.2185334046006837 and parameters: {'degree': 3}. Best is trial 0 with value: 1.2185334046006837.
[I 2024-01-29 17:02:24,555] Trial 4 finished with value: 1.2185334046006837 and parameters: {'degree': 2}. Best is trial 0 with value: 1.2185334046006837.
[I 2024-01-29 17:02:26,671] Trial 5 finished with value: 1.2185334046006837 and parameters: {'degree': 4}. Best is trial 0 with va

In [ ]:
best_model2 = SVC(degree= study2.best_params['degree'])
best_model2.fit(X_train, y_train)

SVC(degree=4)

In [ ]:
y_pred = best_model2.predict(X_test)
accuracy2 = accuracy_score(y_true= y_test, y_pred= y_pred)
print(accuracy2)

0.7623574144486692


In [ ]:
tn, fp, fn, tp = confusion_matrix(y_pred, y_test).ravel()

ACC = (tp + tn)/(tp + tn + fn + fp)
SEN = tp/(tp + fn)
SPE = tn/(tn + fp)

print(f'Accuracy = {ACC}')
print(f'Sensitivity = {SEN}')
print(f'Specificity = {SPE}')


Accuracy = 0.7623574144486692
Sensitivity = 0.746268656716418
Specificity = 0.7790697674418605


In [ ]:
model_name = f"SVM_MACCS_model_Skin_Irritaion{datetime.datetime.now().isoformat()}"
model_path = "../models/"
file = os.path.join(model_path, model_name)
pickle.dump(best_model2, open(file, 'wb'))